> **WARNING** It is more than recommended to work on a copy of that file (_i.e._ not IN the cloned git repository) otherwise next time you update the repo with `git pull` you'll face a `merging` issue and these are tricky to manage

# A first Foreland Basin

In this notebook, we will use the basic `trackscape` module, aka without the python helper, to run fluvial landscapes going through an uplifting range and a foreland basin. It introduces the notion of spatially variable uplift field as well as the monitoring of sediment height.

## Setting up the Parameters

We will set the basic parameters first, see previous notebook for more details. Note that we elongate the model in the Y direction to make a nicely outlined foreland. We also set the left and right BC to periodic in order to have an infinite foreland.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import popscape_dagger as psc
import dagger


%matplotlib widget



ny,nx = 512,256
dy,dx = 200,200
dt = 500
Kr = 1e-5
Ks = 2e-5
dep = 4
rshp = (ny,nx)

# Initialising an empty model in the variable ts
ts = psc.trackscape()
# Initialising the topography and its dimensions
ts.init_random(psc.NOISE.WHITE, nx, ny,dx,dy,"periodic_EW")

# FUnctions to set parameters as global homogeneous values (if not initialised, there is a default value)
ts.set_single_Kr(Kr)
ts.set_single_Ks(Ks)
ts.set_single_depcoeff(dep)

# Deactivate hilllsopes processes
ts.hillslopes_off()

### Setting the uplift field

As mentioned, we will set a simple uplift field to have a range and a foreland. We will proceed in multiple steps:

- Create a 2D array of model size to 0 (shape is ny,nx for n rows and n columns)
- set the central part of the domain in the y direction to 1 mm/yrs of uplift
- visualise
- Linearise (the `c++` model only takes 1D arrays, this will be simplified with a `python` package on the top)

In [ ]:
# initialising the array
Ufield = np.zeros(rshp)

# Getting the coordinates of the third of the domain:
third = round(ny/3)

# Setting to 1 mm/yrs
Ufield[third:2*third,:] = 1e-3

fig,ax = plt.subplots(figsize = (4,5))
cb = ax.imshow(Ufield, cmap = "magma")
ax.set_xlabel("cols")
ax.set_ylabel("rows")
plt.colorbar(cb)
plt.tight_layout()

# Linearising
Ufield = Ufield.ravel()

## Running the model

Let's run the model. You can set ndt to 0 to skip that part, or even jump to the next section, or finally use the stop button at the top of the script to stop the model execution without stopping the all notebook.

This first cell generates the figure, we will plot the topography as well as the sediment height. We are also getting the hillshaded relief, as it does make things nicer and show a reference point for the sediment field.

In [ ]:
fig,ax = plt.subplots(1,2, figsize = (8,5))

topo = ts.get_topo().reshape(rshp)
hillshade = ts.connector.get_HS(ts.get_topo()).reshape(rshp)
topoplot = ax[0].imshow(topo, cmap = "gist_earth")
hsplot0 = ax[0].imshow(hillshade, cmap = "gray", alpha = 0.5, vmin = 0, vmax = 1)
sedplot = ax[1].imshow(np.zeros_like(topo), cmap = "Oranges")
hsplot1 = ax[1].imshow(hillshade, cmap = "gray", alpha = 0.5, vmin = 0, vmax = 1)
plt.colorbar(topoplot, label = "elevation (m)")
plt.colorbar(sedplot, label = "sediment height (m)")
plt.tight_layout()

**Let's now run the model `ndt` times, and update the figure every `nupdate` timesteps** (updating the figure too often can be costly).

You can also set the cmaps value to a constant value with the `set_clim` functions.

> **Note** the figure is updated in place, just above. If you deactivated the widget, it won't work

In [ ]:
ndt = 10000
nupdate = 100

# Main loop
for i in range(ndt):
    # Calling the run function for single flow
    ts.run_SFD(dt)
    # Calling hte block uplift funtion)
    ts.external_uplift(Ufield,dt, False)
    
    # If nupdate^th timestep: I update the fig
    if(i%nupdate == 0):
        ## printing the timestep
        print("                   ",end = "\r")
        print("Timestep",i,end = "\r")
        
        #GEtting the topography
        topo = ts.get_topo().reshape(rshp)
        # Calculating hillshade
        hillshade = ts.connector.get_HS(ts.get_topo()).reshape(rshp)
        # Setting the new data
        topoplot.set_data(topo)
        # Setting the new colorbar limits
        topoplot.set_clim(topo.min(), topo.max())
        # Updating hillshades
        hsplot0.set_data(hillshade)
        hsplot1.set_data(hillshade)
        # Accessing AND updating sed height
        tsed = ts.get_h_sed().reshape(rshp)
        sedplot.set_data(tsed)
        sedplot.set_clim(tsed.min(), tsed.max())
        
        # Apply the update
        fig.canvas.draw()
        
        
        
        